In [1]:
import os
from glob import glob
import numpy as np

from model.VAE import VariationalAutoencoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from tensorflow.compat.v1 import ConfigProto,InteractiveSession
import tensorflow as tf

tf.compat.v1.disable_eager_execution()
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        config = ConfigProto()
        config.gpu_options.allow_growth = True
        session = InteractiveSession(config=config)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [6]:
RUN_FOLDER = 'run/vae/faces'

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER,'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'load' 

DATA_FOLDER = './data/celeba/'

In [4]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMG = len(filenames)

In [5]:
data_generator = ImageDataGenerator(rescale=1./255)

data_flow = data_generator.flow_from_directory(DATA_FOLDER, 
                                               target_size=INPUT_DIM[:2],
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               class_mode='input',
                                               subset="training"
                                              )

Found 202599 images belonging to 1 classes.


In [7]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
vae.encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 64, 32)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [9]:
vae.decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 200)]             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              823296    
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0   

In [17]:
LEARNING_RATE = 0.001
R_LOSS_FACTOR = 10000
EPOCHS = 52
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 47

In [12]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [18]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMG / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 48/52
6332/6331 [==============================] - ETA: 0s - loss: 218.2378 - vae_r_loss: 159.2698 - vae_kl_loss: 58.9642
Epoch 00048: saving model to run/vae/faces/weights/weights-048-218.23.h5

Epoch 00048: saving model to run/vae/faces/weights/weights.h5
6332/6331 [==============================] - 398s 63ms/step - loss: 218.2378 - vae_r_loss: 159.2698 - vae_kl_loss: 58.9642 - lr: 5.0000e-04
Epoch 49/52
6332/6331 [==============================] - ETA: 0s - loss: 218.3505 - vae_r_loss: 159.3315 - vae_kl_loss: 59.0158- ETA: 3s - loss: 218.
Epoch 00049: saving model to run/vae/faces/weights/weights-049-218.35.h5

Epoch 00049: saving model to run/vae/faces/weights/weights.h5
6332/6331 [==============================] - 396s 63ms/step - loss: 218.3505 - vae_r_loss: 159.3315 - vae_kl_loss: 59.0158 - lr: 5.0000e-04
Epoch 50/52
6332/6331 [==============================] - ETA: 0s - loss: 218.1849 - vae_r_loss: 159.1906 - vae_kl_loss: 58.9855
Epoch 00050: saving model to run/vae/faces

KeyboardInterrupt: 